# How to repeat steps for different parameters

* **Difficulty level**: easy
* **Time need to lean**: 10 minutes or less
* **Key points**:
  * `for_each` runs the substep with different parameters
  

Option `for_each` allows you to create substeps with different values of one or more variables.

## `for_each` with names of variables

If you have a variable that have already been defined with a list of values, you can specify its name to parameter `for_each`. **The iteration variable will have a name prefixed with an underscore**.

For eample, option `for_each='method'` creates two substeps with `m1` and `m2`, presented by variable `_method`: 

In [1]:
!touch file1 file2
%run -v1

method = ['m1', 'm2']
input: 'file1', 'file2', for_each='method'
print(f"{_index}: {_input} {_method}")

0: file1 file2 m1


1: file1 file2 m2


Nested loops are also allowed. For example,

In [2]:
!touch file1 file2
%run -v1
[0]
method = ['m1', 'm2']
pars = [1, 2]
input: 'file1', 'file2', for_each=['method', 'pars']
print(f"{_index}: _input={_input} _method={_method}, _pars={_pars}")

0: _input=file1 file2 _method=m1, _pars=1


1: _input=file1 file2 _method=m2, _pars=1


2: _input=file1 file2 _method=m1, _pars=2


3: _input=file1 file2 _method=m2, _pars=2


If you would like to loop the process with several parameters, you can put them into the same level by 'var1,var2'. For example,

In [3]:
!touch file1 file2
%run -v1

[0]
method = ['m1', 'm2']
pars = [1, 2]
input: 'file1', 'file2', for_each=['method,pars']
print(f"{_index}: _input={_input} _method={_method}, _pars={_pars}")

0: _input=file1 file2 _method=m1, _pars=1


1: _input=file1 file2 _method=m2, _pars=2


The variable passed to option `for_each` can a sequence (`list`, `tuple`, `set`, etc), a Pandas `Series`, `Index`, or `DataFrame`. In the last case, each `_loop` variable presents a line in the dataframe and you can access single values using format `_loop["header"]`. For example

In [4]:
%preview df
import pandas as pd
df = pd.DataFrame([(1, 2, 'Hello'), (2, 4, 'World')], columns=['A', 'B', 'C'])

In [5]:
input: for_each='df'
output: f"{_df['A']}_{_df['B']}_{_df['C']}.txt"
print(f'_index={_index}\n_df=\n{_df}\n_output={_output}\n')
sh: expand=True
    touch {_output}

_index=0
_df=
A        1
B        2
C    Hello
Name: 0, dtype: object
_output=1_2_Hello.txt



_index=1
_df=
A        2
B        4
C    World
Name: 1, dtype: object
_output=2_4_World.txt



## `for_each` with dictionary of variables

If you would like define your own loop variable, or if the default loop variable does not work (e.g. loop through `obj.sequence` where `_obj.sequence` is not a valid variable name), you can use a dictionary syntax in the format of `{'varname': sequence}`. Mult-variable and nested loops can be specified in the format of `{'var1': seq1, 'var2': seq2}` (same level) and `[{'var1': seq1}, {'var2': seq2}]` for different levels. 

For example, the first example for this parameter can be written as

In [6]:
!touch file1 file2

input: 'file1', 'file2', for_each=dict(method=['m1', 'm2'])
print(f"{_index}: {_input} {method}")

0: file1 file2 m1


1: file1 file2 m2


and a latter example can be written as

In [7]:
!touch file1 file2
input: 'file1', 'file2', 
   for_each=dict(method=['m1','m2'], pars=[1, 2])
print(f"{_index}: _input={_input} method={method}, pars={pars}")

0: _input=file1 file2 method=m1, pars=1


1: _input=file1 file2 method=m2, pars=2


If you need to defined nested loops, you can use

In [8]:
!touch file1 file2
input: 'file1', 'file2', 
   for_each=[dict(method=['m1','m2']), dict(pars=[1, 2])]
print(f"{_index}: _input={_input} method={method}, pars={pars}")

0: _input=file1 file2 method=m1, pars=1


1: _input=file1 file2 method=m2, pars=1


2: _input=file1 file2 method=m1, pars=2


3: _input=file1 file2 method=m2, pars=2


The dictionary syntax also supports multiple keys. This helps customizing groups of variables. For example in the script below we only care for situations where `n` is greater than `p`,  

In [9]:
!touch a.txt
import itertools
parameter: n = [100, 300]
parameter: p = [50, 100, 200]
parameter: outfile = ['1.txt', '2.txt', '3.txt', '4.txt', '5.txt', '6.txt']
input: 'a.txt', for_each= {'_n,_p': [(_n,_p) for _n in n for _p in p if _n > _p]}
print(f"{_index} {outfile[_index]} {_n} {_p}")

0 1.txt 100 50


1 2.txt 300 50


2 3.txt 300 100


3 4.txt 300 200


## `for_each` and `group_by`

Options `for_each` and `group_by` can be used together, in which case `for_each` is applied to each substep created by `group_by`, creating more substeps.

In [10]:
!touch file1 file2
input: 'file1', 'file2', group_by=1,
   for_each=dict(method=['m1','m2'], pars=[1, 2])
print(f"{_index}: _input={_input} method={method}, pars={pars}")

0: _input=file1 method=m1, pars=1


1: _input=file2 method=m1, pars=1


2: _input=file1 method=m2, pars=2


3: _input=file2 method=m2, pars=2


## Further reading

* [`input` statement](input_statement.html)
* [`for_each` input option](for_each.html)